In [ ]:
import cv2
import picture as pic
import classify as cf
import numpy as np
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import time

base = BaseOverlay("base.bit")

# monitor configuration: 640*480 @ 60Hz   ##配置hdmi显示参数
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

# camera (input) configuration  ##配置显示窗口大小和框图位置
frame_in_w = 640#640
frame_in_h = 480#480
x0 = 300
y0 = 100
width, height = 300, 300

videoIn = cv2.VideoCapture(0) ##    ##开摄像头并设置参数
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))

#font = cv2.FONT_HERSHEY_SIMPLEX #设置字体
#size = 0.5 #设置大小
#width, height = 300, 300 #设置拍摄窗口大小
#x0,y0 = 300, 100 #设置选取位置
#cnt = 1
#cap = cv2.VideoCapture(0) #开摄像头



if __name__ == "__main__":
    while(1):
        #flag, frame = cap.read() #读取摄像头的内容
        flag, frame_vga = videoIn.read()##ret-->flag    ##读摄像头

        if videoIn.isOpened() == 0 : #flag
            time.sleep(3)
            #raise RuntimeError("Error while reading from camera.")

        #frame_vga = cv2.flip(frame_vga, 2)
        roi, res, ret, fourier_result = pic.binaryMask(frame_vga, x0, y0, width, height) #取手势所在框图并进行处理
        
        outframe = hdmi_out.newframe()
        outframe[:] = frame_vga #frame_vga
        hdmi_out.writeframe(outframe)
        #cv2.imshow("roi", roi)  # 显示手势框图
        #cv2.imshow("res", res)
        #cv2.imshow("ret", ret)
        #cv2.imshow('frame', frame) #播放摄像头的内容
        
        key_p=base.buttons[0].read()
        key_q=base.buttons[3].read()
        if key_q == 1:
            break
        elif key_p ==1:
   
            descirptor_in_use = abs(fourier_result)
            fd_test = np.zeros((1,31))  ####31
            temp = descirptor_in_use[1]
            for k in range(1,len(descirptor_in_use)):
                fd_test[0,k-1] = int(100 * descirptor_in_use[k] / temp)
            test_svm = cf.test_fd(fd_test)###################

            gesture = chr(test_svm[0] + 96)
            print("Gesture =",gesture)

    #cap.release()
    #cv2.destroyAllWindows() #关闭所有窗口

    videoIn.release()
    hdmi_out.stop()
    del hdmi_out


capture device is open: True
Gesture = a
Gesture = b
Gesture = f
Gesture = i
Gesture = g
Gesture = f
Gesture = h
Gesture = h
Gesture = g
Gesture = g
Gesture = i
Gesture = d
Gesture = d
Gesture = d
Gesture = c
Gesture = c
Gesture = c
Gesture = c
Gesture = j
Gesture = j
